# Training Phase
This code trains the model. It:

* Uses transfer learning on ResNet50.

* Only fine-tunes the final fully connected layer (you can change this if you want to fine-tune the entire model).

* Tracks training/validation loss and accuracy.

* Saves the best model based on validation loss.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Updated ResNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Directories
data_dir = 'cleaned_and_ready_for_use_data'
train_dir = f"{data_dir}/train"
val_dir = f"{data_dir}/val"
test_dir = f"{data_dir}/test"

# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# Loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# ✅ Unfreeze last few layers
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005)

# Optional learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

# Training
epochs = 10
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss, correct_val, total_val = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_acc = correct_val / total_val
    val_loss /= len(val_loader)

    # Adjust LR
    scheduler.step(val_loss)

    print(f"\nEpoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        torch.save(model.state_dict(), "best_model.pth")
        best_val_loss = val_loss
        print("✅ Best model saved.")

print("\n🎉 Training complete!")



/home/fredrick-gacheru/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fredrick-gacheru/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch [1/10]
Train Loss: 0.0636, Accuracy: 0.9769
Val Loss: 0.0038, Accuracy: 0.9989
✅ Best model saved.

Epoch [2/10]
Train Loss: 0.0260, Accuracy: 0.9919
Val Loss: 0.0063, Accuracy: 0.9983

Epoch [3/10]
Train Loss: 0.0249, Accuracy: 0.9919
Val Loss: 0.0055, Accuracy: 0.9983

Epoch [4/10]
Train Loss: 0.0196, Accuracy: 0.9940
Val Loss: 0.0027, Accuracy: 0.9994
✅ Best model saved.

Epoch [5/10]
Train Loss: 0.0161, Accuracy: 0.9949
Val Loss: 0.0035, Accuracy: 0.9994

Epoch [6/10]
Train Loss: 0.0083, Accuracy: 0.9972
Val Loss: 0.0034, Accuracy: 0.9994

Epoch [7/10]
Train Loss: 0.0069, Accuracy: 0.9979
Val Loss: 0.0058, Accuracy: 0.9989

Epoch [8/10]
Train Loss: 0.0045, Accuracy: 0.9984
Val Loss: 0.0060, Accuracy: 0.9989

Epoch [9/10]
Train Loss: 0.0035, Accuracy: 0.9992
Val Loss: 0.0082, Accuracy: 0.9989

Epoch [10/10]
Train Loss: 0.0023, Accuracy: 0.9991
Val Loss: 0.0067, Accuracy: 0.9989

🎉 Training complete!


# Testing the model
I am testing the model. 

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Normalization values (same as training)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Test transform
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Dataset & Loader
data_dir = 'cleaned_and_ready_for_use_data'
test_dir = f"{data_dir}/test"
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Recreate model architecture
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # Must match training
model = model.to(device)

# Load trained weights
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()

# Evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))

print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



/home/fredrick-gacheru/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fredrick-gacheru/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Classification Report:
              precision    recall  f1-score   support

         cat       1.00      1.00      1.00       115
         dog       1.00      1.00      1.00       329
       human       1.00      1.00      1.00      1324

    accuracy                           1.00      1768
   macro avg       1.00      1.00      1.00      1768
weighted avg       1.00      1.00      1.00      1768

Confusion Matrix:
[[ 115    0    0]
 [   0  329    0]
 [   0    0 1324]]


# Analysis
The test results are slightly worrying because they seem to imply that the model had access to the test data in the training phase. I will use different images. 

In [19]:
from PIL import Image
import torch
import torchvision.transforms as transforms
import cv2

# Load YOLOv5 model only once (outside the function)
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.conf = 0.5  # Confidence threshold
LABEL_MAP = {'cat': 15, 'dog': 16, 'human': 0}  # COCO labels

def preprocess_with_yolo(image_path, resize=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"❌ Failed to load image: {image_path}")
    
    results = yolo_model(image)
    detections = results.xyxy[0]

    for *box, conf, cls in detections:
        if int(cls) in LABEL_MAP.values():
            x1, y1, x2, y2 = map(int, box)
            cropped = image[y1:y2, x1:x2]
            resized = cv2.resize(cropped, resize)
            return Image.fromarray(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for PIL

    print(f"⚠️ No cat or dog confidently detected in: {image_path}")
    return None

def predict_image(image_path, model_path, class_names):
    # Define the transform (must be before using it!)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    # Preprocess with YOLOv5
    preprocessed_image = preprocess_with_yolo(image_path)
    if preprocessed_image is None:
        return "something else"

    # Apply transform to preprocessed PIL image
    image_tensor = transform(preprocessed_image).unsqueeze(0).to(device)

    # Load model
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 4)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Predict
    with torch.no_grad():
        outputs = model(image_tensor)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)
        top_prob, top_class = torch.max(probs, 0)

    # Print probabilities
    print(f"\nPrediction results for: {image_path}")
    for i, prob in enumerate(probs):
        label = class_names[i] if i < len(class_names) else "something else"
        print(f"{label}: {prob.item()*100:.2f}%")

    predicted_label = class_names[top_class.item()] if top_class.item() < len(class_names) else "something else"
    return predicted_label



Using cache found in /home/fredrick-gacheru/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-12 Python-3.12.7 torch-2.7.0+cpu CPU

Fusing layers... 
[W728 01:27:22.173915654 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.175689076 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.176180582 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.176574982 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.177532447 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.177996188 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.178437537 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.178985341 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:22.179845855 NNPACK.cpp:57] Could not

In [20]:
class_names = ['cat', 'dog', 'human', 'something else']
label = predict_image("demo_images/human_1.jpg", "best_model.pth", class_names)
print(f"Predicted label: {label}")

# for i in range(1, 6):
#     print(f"\n--- Predicting dog_{i}.jpg ---")
#     predict_image(f"demo_images/dog_{i}.jpg", model, val_test_transforms, class_names)

/home/fredrick-gacheru/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



Prediction results for: demo_images/human_1.jpg
cat: 38.01%
dog: 61.77%
human: 0.17%
something else: 0.05%
Predicted label: dog


In [21]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import cv2

# Load YOLO model once
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.conf = 0.5
LABEL_MAP = {'cat': 15, 'dog': 16, 'human': 0}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Functions ----------

def preprocess_with_yolo(image_path, resize=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"❌ Failed to load image: {image_path}")

    results = yolo_model(image)
    detections = results.xyxy[0]

    for *box, conf, cls in detections:
        if int(cls) in LABEL_MAP.values():
            x1, y1, x2, y2 = map(int, box)
            cropped = image[y1:y2, x1:x2]
            resized = cv2.resize(cropped, resize)
            return Image.fromarray(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    return None


def load_model(model_path, num_classes=4):
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model


def predict_image(image_path, model, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    preprocessed_image = preprocess_with_yolo(image_path)
    if preprocessed_image is None:
        return "something else", None

    image_tensor = transform(preprocessed_image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        probs = torch.nn.functional.softmax(outputs[0], dim=0)
        top_prob, top_class = torch.max(probs, 0)

    predicted_label = class_names[top_class.item()]
    return predicted_label, probs.cpu().numpy()


# ---------- Main (for testing only) ----------
if __name__ == "__main__":
    model = load_model("best_model.pth", num_classes=4)
    class_names = ['cat', 'dog', 'human', 'something else']
    label, probs = predict_image("demo_images/human_1.jpg", model, class_names)
    print(f"Predicted: {label}")
    if probs is not None:
        print("Probabilities per class:")
        for cls_name, p in zip(class_names, probs):
            print(f"  {cls_name}: {p * 100:.1f}%")  # formatted as percentage



Using cache found in /home/fredrick-gacheru/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-12 Python-3.12.7 torch-2.7.0+cpu CPU

Fusing layers... 
[W728 01:27:50.991971339 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.993580376 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.994074535 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.994453575 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.995394608 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.995868798 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.996298958 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.997122594 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W728 01:27:50.998251109 NNPACK.cpp:57] Could not

Predicted: dog
Probabilities per class:
  cat: 38.0%
  dog: 61.8%
  human: 0.2%
  something else: 0.1%
